# Validating against log file example

In [1]:
%load_ext autoreload
%autoreload 2

Run with log level set to `LOG_LEVEL_IO` so that all IO oeprations are logged to the file.

In [2]:
from pathlib import Path
from pylabrobot.config import Config
from pylabrobot.io import LOG_LEVEL_IO

config = Config(
  logging=Config.Logging(
    level=LOG_LEVEL_IO,
    log_dir=Path("my_logs")
  )
)
import pylabrobot
pylabrobot.configure(config)

Instantiate the frontends and backends:

In [3]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.resources.hamilton import STARLetDeck

backend = STAR()
lh = LiquidHandler(backend=backend, deck=STARLetDeck())

Do a real run first, without the validation enabled. This will generate the log file you can later compare against. This effectively ensures you have a real working protocol. The benefit of using log files is whenever you change your protocol and have seen it run, you can just grab the log file and use it for validation. No need to manually write tests.

On validation, before calling setup, run `pylabrobot.validate` to enable the validation. Pass a log file that contains the commands we should check against. This returns a `LogReader` object which you can use at the end to check if the all commands were executed correctly.

In [4]:
import pylabrobot
lr = pylabrobot.validate("./my_logs/pylabrobot-20250204.log")

In [5]:
await lh.setup()

In [6]:
from pylabrobot.resources import TIP_CAR_480_A00, HT

tip_car = TIP_CAR_480_A00(name="tip_car")
tip_car[0] = tr = HT(name="ht")
lh.deck.assign_child_resource(tip_car, rails=1)

In [7]:
await lh.pick_up_tips(tr["A1:H1"])

In [8]:
await lh.return_tips()

Use `lr.done()` to check if all commands were executed.

In [9]:
lr.done()

Validation successful!


In [10]:
await lh.stop()

## Example: wrong log

In [11]:
# reset the log validator
lr.reset()

Start with clean instances to prevent caching issues. (This is just for the sake of demonstration.)

In [12]:
backend = STAR()
lh = LiquidHandler(backend=backend, deck=STARLetDeck())

lr = pylabrobot.validate("./my_logs/pylabrobot-20250204.log")

await lh.setup()

tip_car = TIP_CAR_480_A00(name="tip_car")
tip_car[0] = tr = HT(name="ht")
lh.deck.assign_child_resource(tip_car, rails=1)

When the generated command differs from the log file, a ValidationError will be raised.

In [13]:
await lh.pick_up_tips(tr["A2:H2"])  # column 2 instead of 1

expected: C0TPid0011xp01179 01179 01179 01179 01179 01179 01179 01179yp1458 1368 1278 1188 1098 1008 0918 0828tm1 1 1 1 1 1 1 1tt01tp2266tz2166th2450td0
actual:   C0TPid0011xp01269 01269 01269 01269 01269 01269 01269 01269yp1458 1368 1278 1188 1098 1008 0918 0828tm1 1 1 1 1 1 1 1tt01tp2266tz2166th2450td0
                        ^^    ^^    ^^    ^^    ^^    ^^    ^^    ^^                                                                                    


ValidationError: Data mismatch: difference was written to stdout.